In [1]:
%reload_ext autoreload
%autoreload 2

import os

os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import math
import numpy as np
import pandas as pd
import datetime
import pytz
import seaborn as sns
from datetime import datetime, timedelta
import concurrent.futures
from itertools import product
from tqdm import tqdm

from watttime import WattTimeForecast, WattTimeHistorical

import data.s3 as s3u
import evaluation.eval_framework as efu

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

actual_data = WattTimeHistorical(username, password)
hist_data = WattTimeForecast(username, password)

s3 = s3u.s3_utils()

In [2]:
regions = [
 'CAISO_NORTH',
 'SPP_TX',
 'ERCOT_EASTTX',
 'FPL',
 'SOCO',
 'PJM_CHICAGO',
 'LDWP',
 'PJM_DC',
 'NYISO_NYC'
]

In [3]:
dates_2023 = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

In [4]:
forecast_generator = WattTimeForecast(username, password)
historical_generator = WattTimeHistorical(username, password)

In [ ]:
def get_daily_historical_data(date, region):
    daily_data = historical_generator.get_historical_pandas(start = date,
                                               end = date + timedelta(days = 1),
                                               region = region,
                                               signal_type="co2_moer")
    
    daily_data["region"] = region

    return daily_data
    
for region in regions:
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        print(f"Getting 2023 actuals data for {region}")

        result = list(tqdm(
            executor.map(lambda date: get_daily_historical_data(date, region), dates_2023),
            total=len(dates_2023),
            desc=f"Processing forecast data for region {region}"
        ))
    
    out = pd.concat(result, ignore_index=True)
    s3.store_parquetdataframe(out, f'complete_2023_actual_history/{region}.parquet')
    print("Wrote parquet file to s3")

Getting 2023 actuals data for CAISO_NORTH


Processing forecast data for region CAISO_NORTH: 100%|██████████| 365/365 [00:02<00:00, 124.34it/s]


Successful S3 put_object response. Status - 200
Wrote parquet file to s3
Getting 2023 actuals data for SPP_TX


Processing forecast data for region SPP_TX: 100%|██████████| 365/365 [00:03<00:00, 106.94it/s]


Successful S3 put_object response. Status - 200
Wrote parquet file to s3
Getting 2023 actuals data for ERCOT_EASTTX


...

Successful S3 put_object response. Status - 200
Wrote parquet file to s3
Getting 2023 actuals data for PJM_DC


Processing forecast data for region PJM_DC: 100%|██████████| 365/365 [00:13<00:00, 26.10it/s]


Successful S3 put_object response. Status - 200
Wrote parquet file to s3
Getting 2023 actuals data for NYISO_NYC


Processing forecast data for region NYISO_NYC: 100%|██████████| 365/365 [00:04<00:00, 89.97it/s] 


Successful S3 put_object response. Status - 200
Wrote parquet file to s3


In [ ]:
def get_daily_forecast_data(date, region, horizon = 24):
    daily_data = forecast_generator.get_historical_forecast_pandas(
                start=date, 
                end=date + timedelta(days = 1),
                region=region,
                signal_type="co2_moer",
                horizon_hours=horizon,
            )
    daily_data["region"] = region

    return daily_data


for region in regions:
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        print(f"Getting 2023 data for {region}")

        result = list(tqdm(
            executor.map(lambda date: get_daily_forecast_data(date, region), dates_2023),
            total=len(dates_2023),
            desc=f"Processing forecast data for region {region}"
        ))
    
    out = pd.concat(result, ignore_index=True)
    s3.store_parquetdataframe(out, f'complete_2023_forecast_history/{region}.parquet')
    print("Wrote parquet file to s3")

Getting 2023 data for CAISO_NORTH


Processing forecast data for region CAISO_NORTH:   0%|          | 0/365 [00:00<?, ?it/s]


 Warnings Returned: {'region': 'CAISO_NORTH', 'signal_type': 'co2_moer', 'horizon_hours': 24, 'start': Timestamp('2023-01-19 00:00:00+0000', tz='UTC'), 'end': Timestamp('2023-01-20 00:00:00+0000', tz='UTC')} {'data_point_period_seconds': 300, 'region': 'CAISO_NORTH', 'signal_type': 'co2_moer', 'units': 'lbs_co2_per_mwh', 'warnings': [{'type': 'MISSING_DATA', 'message': 'You made a valid request, but we have some datapoints missing. This can happen for a variety of reasons. If the data is less than a week old, we will most likely fill in this data soon. If it is older than that, we are missing enough important grid data during this time period that we are not able to generate an accurate estimate'}], 'model': {'date': '2023-03-01'}, 'generated_at_period_seconds': 300}

 Warnings Returned: {'region': 'CAISO_NORTH', 'signal_type': 'co2_moer', 'horizon_hours': 24, 'start': Timestamp('2023-01-20 00:00:00+0000', tz='UTC'), 'end': Timestamp('2023-01-21 00:00:00+0000', tz='UTC')} {'data_point

Processing forecast data for region CAISO_NORTH:  25%|██▌       | 92/365 [03:26<05:18,  1.17s/it]   


 Warnings Returned: {'region': 'CAISO_NORTH', 'signal_type': 'co2_moer', 'horizon_hours': 24, 'start': Timestamp('2023-05-24 00:00:00+0000', tz='UTC'), 'end': Timestamp('2023-05-25 00:00:00+0000', tz='UTC')} {'data_point_period_seconds': 300, 'region': 'CAISO_NORTH', 'signal_type': 'co2_moer', 'units': 'lbs_co2_per_mwh', 'warnings': [{'type': 'MISSING_DATA', 'message': 'You made a valid request, but we have some datapoints missing. This can happen for a variety of reasons. If the data is less than a week old, we will most likely fill in this data soon. If it is older than that, we are missing enough important grid data during this time period that we are not able to generate an accurate estimate'}], 'model': {'date': '2023-03-01'}, 'generated_at_period_seconds': 300}

 Warnings Returned: {'region': 'CAISO_NORTH', 'signal_type': 'co2_moer', 'horizon_hours': 24, 'start': Timestamp('2023-06-02 00:00:00+0000', tz='UTC'), 'end': Timestamp('2023-06-03 00:00:00+0000', tz='UTC')} {'data_point

...

Processing forecast data for region FPL:  57%|█████▋    | 208/365 [06:59<10:18,  3.94s/it]


 Warnings Returned: {'region': 'FPL', 'signal_type': 'co2_moer', 'horizon_hours': 24, 'start': Timestamp('2023-11-03 00:00:00+0000', tz='UTC'), 'end': Timestamp('2023-11-04 00:00:00+0000', tz='UTC')} {'data_point_period_seconds': 300, 'region': 'FPL', 'signal_type': 'co2_moer', 'units': 'lbs_co2_per_mwh', 'warnings': [{'type': 'MISSING_DATA', 'message': 'You made a valid request, but we have some datapoints missing. This can happen for a variety of reasons. If the data is less than a week old, we will most likely fill in this data soon. If it is older than that, we are missing enough important grid data during this time period that we are not able to generate an accurate estimate'}], 'model': {'date': '2022-10-01'}, 'generated_at_period_seconds': 300}


Processing forecast data for region FPL: 100%|██████████| 365/365 [09:45<00:00,  1.60s/it]


Successful S3 put_object response. Status - 200
Wrote parquet file to s3
Getting 2023 data for SOCO


Processing forecast data for region SOCO:   0%|          | 0/365 [00:00<?, ?it/s]